In [23]:
from StarbucksAnalysis.model import pretty_coefficients, expand_grid, fit_logistic, model_outcome
import pandas as pd
import numpy as np
import pickle
import patsy

# Load data

In [11]:
offers = pickle.load(open("processed_data/offers_cleaned.p", "rb"))

# No interaction

In [12]:
lg_no_interaction, formula, exog_train, exog_val, _,  y_val = fit_logistic(offers, ["income_thousands", "gender", "young", "seniority", "social"])

Optimization terminated successfully.
         Current function value: 0.617162
         Iterations 5


In [13]:
model_outcome(lg_no_interaction, exog_val, y_val)

Accuracy:  0.6658897827231035

Confusion matrix
[[5984 1560]
 [2884 2873]]

Scoring
              precision    recall  f1-score   support

           0       0.67      0.79      0.73      7544
           1       0.65      0.50      0.56      5757

    accuracy                           0.67     13301
   macro avg       0.66      0.65      0.65     13301
weighted avg       0.66      0.67      0.66     13301



In [14]:
lg_no_interaction.summary().tables[0]

Dep. Variable:,completed,No. Observations:,53200
Model:,Logit,Df Residuals:,53190
Method:,MLE,Df Model:,9
Date:,"Sat, 25 Mar 2023",Pseudo R-squ.:,0.09736
Time:,12:27:20,Log-Likelihood:,-32833.
converged:,True,LL-Null:,-36375.
Covariance Type:,nonrobust,LLR p-value:,0.000


In [15]:
pretty_coefficients(lg_no_interaction.summary(), exog_train)

,Variable,coef,odds ratio,z,P>|z|
0,Intercept,-0.770,0.463,-15.169,0.000
1,[T.Bogo],-0.546,0.579,-20.757,0.000
2,[T.Discount],-0.053,0.948,-2.060,0.039
3,gender[T.Male],-0.198,0.820,-10.048,0.000
4,gender[T.Other],0.377,1.458,4.623,0.000
5,young[T.True],-0.264,0.768,-9.521,0.000
6,seniority[T.Gen2],0.358,1.431,10.531,0.000
7,seniority[T.Gen3],-0.640,0.527,-19.178,0.000
8,social[T.True],1.222,3.393,59.477,0.000
9,income_thousands,0.004,1.004,9.464,0.000


# Interaction

In [16]:

lg_interaction, formula, exog_train, exog_val, _, y_val = fit_logistic(offers, ["income_thousands", "gender", "young", "seniority", "social"], interaction = True)

Optimization terminated successfully.
         Current function value: 0.603956
         Iterations 5


In [17]:
pretty_coefficients(lg_interaction.summary(), exog_train)

,Variable,coef,odds ratio,z,P>|z|
0,Intercept,0.453,1.573,4.366,0.000
1,[T.Bogo],-1.990,0.137,-15.318,0.000
2,[T.Discount],-1.445,0.236,-11.078,0.000
3,gender[T.Male]:[Bogo],-0.339,0.712,-11.016,0.000
4,gender[T.Other]:[Bogo],0.472,1.603,3.660,0.000
5,gender[T.Male]:[Discount],-0.163,0.849,-4.973,0.000
6,gender[T.Other]:[Discount],0.403,1.496,2.929,0.003
7,gender[T.Male]:[Informational],-0.012,0.988,-0.270,0.787
8,gender[T.Other]:[Informational],0.178,1.194,1.038,0.299
9,young[T.True]:[Bogo],-0.326,0.722,-7.233,0.000


In [9]:
model_outcome(lg_interaction, exog_val, y_val)

Accuracy:  0.674009472971957

Confusion matrix
[[5878 1666]
 [2670 3087]]

Scoring
              precision    recall  f1-score   support

           0       0.69      0.78      0.73      7544
           1       0.65      0.54      0.59      5757

    accuracy                           0.67     13301
   macro avg       0.67      0.66      0.66     13301
weighted avg       0.67      0.67      0.67     13301



There is an issue with patsy: it can not be save, in some circunstances, using the pickle module. The fitted model must be use for some visualizations in the notebook number for. The model will just be fitted again using the same code

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report, confusion_matrix
import statsmodels.api as sm
import patsy
import itertools
import pandas as pd
import numpy as np


In [26]:

    # sets Informational as base category for offer_type
    base_category = 'Informational'
    offer_base = "C(offer_type, Treatment('{0}'))".format(base_category)

    # construtio of the formula for the model
    join_string =":{0} + "
    formula = join_string.join(["income_thousands", "gender", "young", "seniority", "social"])

    formula += ":{0}"

    formula = formula.format(offer_base)

    # inclusion of dependent and offer_type to create train and tests sets
    model_data = offers[["completed"] + ["offer_type"] + ["income_thousands", "gender", "young", "seniority", "social"]]

    train, val = train_test_split(model_data, test_size=0.2, random_state=123)

    # obtaining data for the model from formulas
    y_train = train["completed"]
    X_train = train.drop("completed", axis=1)

    y_val = val["completed"]
    X_val = val.drop("completed", axis=1)

    exog_train = patsy.dmatrix(formula, data=X_train)
    exog_val = patsy.dmatrix(formula, data=X_val)

    # fit model
    lg = sm.Logit(endog=y_train, exog=exog_train)
    result = lg.fit()


Optimization terminated successfully.
         Current function value: 0.603956
         Iterations 5


In [27]:
formula

"income_thousands:C(offer_type, Treatment('Informational')) + gender:C(offer_type, Treatment('Informational')) + young:C(offer_type, Treatment('Informational')) + seniority:C(offer_type, Treatment('Informational')) + social:C(offer_type, Treatment('Informational'))"

In [28]:
# sets Informational as base category for offer_type
base_category = 'Informational'
offer_base = "C(offer_type, Treatment('{0}'))".format(base_category)

# construtio of the formula for the model
join_string = ":C(offer_type, Treatment('{0}')) + ".format(base_category)
formula = join_string.join(["income_thousands", "gender", "young", "seniority", "social"])

# inclusion of dependent and offer_type to create train and tests sets
model_data = offers[["completed"] + ["offer_type"] + ["income_thousands", "gender", "young", "seniority", "social"]]

train, val = train_test_split(model_data, test_size=0.2, random_state=123)

# obtaining data for the model from formulas
y_train = train["completed"]
X_train = train.drop("completed", axis=1)

y_val = val["completed"]
X_val = val.drop("completed", axis=1)

exog_train = patsy.dmatrix(formula, data=X_train)
exog_val = patsy.dmatrix(formula, data=X_val)

# fit model
lg = sm.Logit(endog=y_train, exog=exog_train)
result = lg.fit()

Optimization terminated successfully.
         Current function value: 0.610088
         Iterations 5
